In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [23]:
tariff_events = {
    "2018-01-22": "Solar and washing machine tariffs announced",
    "2018-03-01": "Steel and aluminum tariffs announced",
    "2018-03-08": "Steel and aluminum tariffs signed",
    "2018-03-22": "$50B China tariffs (Section 301 memo)",
    "2018-04-03": "USTR publishes list of 1,300+ Chinese goods",
    "2018-04-04": "China retaliates with tariffs on 106 US products",
    "2018-05-31": "Canada, Mexico, EU hit with steel/aluminum tariffs",
    "2018-06-15": "$34B in Chinese goods to face tariffs (start Jul 6)",
    "2018-07-06": "$34B in Chinese tariffs go into effect",
    "2018-08-01": "Trump threatens 25% on $200B Chinese goods",
    "2018-08-23": "$16B in China tariffs go into effect",
    "2018-09-17": "$200B in China goods to get 10% tariff (List 3)",
    "2018-12-01": "Trump-Xi G20 truce (90-day pause)",
    "2019-05-10": "Tariffs raised from 10% to 25% on $200B goods",
    "2019-08-01": "10% tariffs on $300B announced (List 4A/B)",
    "2019-09-01": "First part of $300B tariffs implemented (List 4A)",
    "2019-12-02": "Steel/aluminum tariffs on Brazil and Argentina",
    "2019-12-15": "List 4B tariffs suspended (Phase One deal)",
    "2020-01-15": "Phase One trade deal signed"
}

second_term_events = {
    "2025-02-01": "Tariffs on Canada, Mexico, China (25%, 10%) over drug trafficking concerns",
    "2025-03-03": "Proceeding with tariffs on Canada and Mexico under IEEPA",
    "2025-03-04": "Tariffs on all Chinese imports raised to 20%",
    "2025-03-12": "Global 25% tariff on steel and aluminum implemented",
}

#### SP Index.

In [24]:
df_SP = pd.read_csv('SP500.csv')
df_SP['observation_date'] = pd.to_datetime(df_SP['observation_date'])
df_SP.set_index('observation_date', inplace=True)

combined_events = {**tariff_event_dates, **{datetime.strptime(k, "%Y-%m-%d"): v for k, v in second_term_events.items()}}

results = []

for event_date, event_desc in combined_events.items():
    try:
        before = df_SP[:event_date].iloc[-1]
        after = df_SP[event_date:].iloc[1]  
        pct_change = ((after.SP500 - before.SP500) / before.SP500) * 100
        results.append({
            "Date": event_date.date(),
            "Event": event_desc,
            "S&P Before": before.SP500,
            "S&P After": after.SP500,
            "% Change": round(pct_change, 2)
        })
    except Exception as e:
        results.append({
            "Date": event_date.date(),
            "Event": event_desc,
            "S&P Before": None,
            "S&P After": None,
            "% Change": None,
            "Note": f"Error: {str(e)}"
        })

results_df = pd.DataFrame(results)
results_df

,Date,Event,S&P Before,S&P After,% Change
0,2018-01-22,Solar and washing machine tariffs announced,2832.97,2839.13,0.22
1,2018-03-01,Steel and aluminum tariffs announced,2677.67,2691.25,0.51
2,2018-03-08,Steel and aluminum tariffs signed,2738.97,2786.57,1.74
3,2018-03-22,$50B China tariffs (Section 301 memo),2643.69,2588.26,-2.10
4,2018-04-03,"USTR publishes list of 1,300+ Chinese goods",2614.45,2644.69,1.16
5,2018-04-04,China retaliates with tariffs on 106 US products,2644.69,2662.84,0.69
6,2018-05-31,"Canada, Mexico, EU hit with steel/aluminum tar...",2705.27,2734.62,1.08
7,2018-06-15,$34B in Chinese goods to face tariffs (start J...,2779.66,2773.75,-0.21
8,2018-07-06,$34B in Chinese tariffs go into effect,2759.82,2784.17,0.88
9,2018-08-01,Trump threatens 25% on $200B Chinese goods,2813.36,2827.22,0.49


In [25]:
positive_days = results_df["% Change"].dropna().gt(0).sum()
total_days = results_df["% Change"].dropna().count()
print(f"Market went up {positive_days} out of {total_days} times ({(positive_days / total_days) * 100:.2f}%)")

Market went up 13 out of 23 times (56.52%)


#### US 10 year treasury securities.

In [28]:
df_DG = pd.read_csv('DGS10.csv')
df_DG['observation_date'] = pd.to_datetime(df_DG['observation_date'])
df_DG.set_index('observation_date', inplace=True)

In [33]:
results_DG = []

for event_date, event_desc in combined_events.items():
    try:
        before = df_DG[:event_date].iloc[-1]
        after = df_DG[event_date:].iloc[1]  
        pct_change = ((after.DGS10 - before.DGS10) / before.DGS10) * 100
        results_DG.append({
            "Date": event_date.date(),
            "Event": event_desc,
            "S&P Before": before.DGS10,
            "S&P After": after.DGS10,
            "% Change": round(pct_change, 2)
        })
    except Exception as e:
        results_DG.append({
            "Date": event_date.date(),
            "Event": event_desc,
            "S&P Before": None,
            "S&P After": None,
            "% Change": None,
            "Note": f"Error: {str(e)}"
        })

results_df_DG = pd.DataFrame(results_DG)
results_df_DG

,Date,Event,S&P Before,S&P After,% Change
0,2018-01-22,Solar and washing machine tariffs announced,2.66,2.63,-1.13
1,2018-03-01,Steel and aluminum tariffs announced,2.81,2.86,1.78
2,2018-03-08,Steel and aluminum tariffs signed,2.86,2.90,1.40
3,2018-03-22,$50B China tariffs (Section 301 memo),2.83,2.82,-0.35
4,2018-04-03,"USTR publishes list of 1,300+ Chinese goods",2.79,2.79,0.00
5,2018-04-04,China retaliates with tariffs on 106 US products,2.79,2.83,1.43
6,2018-05-31,"Canada, Mexico, EU hit with steel/aluminum tar...",2.83,2.89,2.12
7,2018-06-15,$34B in Chinese goods to face tariffs (start J...,2.93,2.92,-0.34
8,2018-07-06,$34B in Chinese tariffs go into effect,2.82,2.86,1.42
9,2018-08-01,Trump threatens 25% on $200B Chinese goods,3.00,2.98,-0.67


In [34]:
positive_days_DG = results_df_DG["% Change"].dropna().gt(0).sum()
total_days_DG = results_df_DG["% Change"].dropna().count()
print(f"Market went up {positive_days_DG} out of {total_days_DG} times ({(positive_days_DG / total_days_DG) * 100:.2f}%)")

Market went up 10 out of 23 times (43.48%)
